In [3]:
import re
import torch
from collections import Counter

TRAIN_FILE = '../data/UD_English-EWT/UD_English-EWT/en_ewt-ud-train.conllu'
DEV_FILE = '../data/UD_English-EWT/UD_English-EWT/en_ewt-ud-dev.conllu'

# --- Tokens Đặc biệt ---
# <UNK> (Unknown): cho các từ không có trong từ điển 
# <PAD> (Padding): cho việc đệm chuỗi 
UNK_TOKEN = "<UNK>"
PAD_TOKEN = "<PAD>"


## 1. Viết hàm đọc file .conllu 
def load_conllu(file_path):
    """
    Đọc dữ liệu từ file CoNLL-U
    Trả về danh sách các câu, mỗi câu là danh sách các cặp (word, upos_tag)
    """
    sentences = []
    current_sentence = []

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()

                # Dòng trống ngăn cách giữa các câu 
                if not line:
                    if current_sentence:
                        sentences.append(current_sentence)
                    current_sentence = []
                    continue

                # Bỏ qua các dòng comment (bắt đầu bằng '#')
                if line.startswith('#'):
                    continue

                # Dòng chứa thông tin token, các trường cách nhau bởi tab 
                fields = line.split('\t')

                # Cần ít nhất 4 trường để lấy FORM và UPOS
                if len(fields) >= 4:
                    # Cột 2 (index 1): FORM (Từ gốc)
                    word = fields[1]
                    # Cột 4 (index 3): UPOS (Nhãn Part-of-Speech) 
                    upos_tag = fields[3]
                    
                    # Kiểm tra ID (fields[0]) để đảm bảo không phải node đa từ/node trống
                    if re.match(r'^\d+$', fields[0]):
                        current_sentence.append((word, upos_tag))
            
            # Thêm câu cuối cùng nếu có
            if current_sentence:
                sentences.append(current_sentence)
    
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file tại đường dẫn {file_path}. Vui lòng kiểm tra lại.")
        return []

    return sentences


## 2. Xây dựng Từ điển (Vocabulary) 
def build_vocabulary(train_data):
    """
    Tạo hai từ điển word_to_ix và tag_to_ix từ dữ liệu huấn luyện.
    """
    
    # Khởi tạo từ điển word_to_ix
    # <UNK> phải là 0, <PAD> là 1
    word_to_ix = {UNK_TOKEN: 0, PAD_TOKEN: 1} 
    
    # Khởi tạo từ điển tag_to_ix
    # <PAD> nên là 0 (sẽ được dùng làm ignore_index trong CrossEntropyLoss )
    tag_to_ix = {PAD_TOKEN: 0} 

    unique_words = set()
    unique_tags = set()

    # Thu thập tất cả từ và nhãn UPOS duy nhất
    for sentence in train_data:
        for word, tag in sentence:
            # Chuẩn hóa từ (thường là lowercase để giảm kích thước từ điển)
            unique_words.add(word.lower()) 
            unique_tags.add(tag)

    # Xây dựng word_to_ix (bắt đầu từ index 2)
    next_word_index = 2
    for word in sorted(list(unique_words)):
        word_to_ix[word] = next_word_index
        next_word_index += 1

    # Xây dựng tag_to_ix (bắt đầu từ index 1)
    next_tag_index = 1
    for tag in sorted(list(unique_tags)):
        tag_to_ix[tag] = next_tag_index
        next_tag_index += 1

    return word_to_ix, tag_to_ix

# --- THỰC THI TASK 1 ---

# 1. Tải Dữ liệu
train_data = load_conllu(TRAIN_FILE)
dev_data = load_conllu(DEV_FILE)

# 2. Xây dựng Từ điển
word_to_ix, tag_to_ix = build_vocabulary(train_data)
print("--- KẾT QUẢ TASK 1: Xây dựng Từ điển ---")  
# In ra kích thước của hai từ điển
print(f"* Kích thước từ điển word_to_ix (gồm <UNK>, <PAD>): {len(word_to_ix)}")
print(f"* Kích thước từ điển tag_to_ix (gồm <PAD>): {len(tag_to_ix)}")
print("---------------------------------------")



--- KẾT QUẢ TASK 1: Xây dựng Từ điển ---
* Kích thước từ điển word_to_ix (gồm <UNK>, <PAD>): 16656
* Kích thước từ điển tag_to_ix (gồm <PAD>): 18
---------------------------------------


In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

# Định nghĩa hàm lấy index (phải được định nghĩa ngoài hàm để sử dụng)
def get_index_from_word(word, word_to_ix, unk_token):
    """
    Chuyển đổi từ (word) sang chỉ số (index). 
    Sử dụng UNK_TOKEN nếu từ không có trong từ điển.
    """
    # Chuẩn hóa về chữ thường (như khi xây dựng từ điển)
    return word_to_ix.get(word.lower(), word_to_ix[unk_token])

class POSDataset(Dataset):
    """Lớp Dataset tùy chỉnh cho bài toán POS Tagging."""
    
    def __init__(self, data, word_to_ix, tag_to_ix, unk_token):
        # data: Danh sách các câu, mỗi câu là list[(word, tag)] 
        self.data = data
        self.word_to_ix = word_to_ix
        self.tag_to_ix = tag_to_ix
        self.unk_index = word_to_ix[unk_token]
        
    def __len__(self):
        """Trả về tổng số câu trong bộ dữ liệu."""
        return len(self.data) 
        
    def __getitem__(self, idx):
        """
        Nhận vào một index và trả về một cặp tensor: (sentence_indices, tag_indices).
        """
        sentence = self.data[idx]
        
        # Ánh xạ từ (word) sang chỉ số (index)
        sentence_indices = [
            get_index_from_word(word, self.word_to_ix, UNK_TOKEN)
            for word, tag in sentence
        ]
        
        # Ánh xạ nhãn (tag) sang chỉ số (index)
        tag_indices = [
            self.tag_to_ix[tag]
            for word, tag in sentence
        ]
        
        # Trả về dưới dạng Tensor
        return (torch.tensor(sentence_indices, dtype=torch.long),
                torch.tensor(tag_indices, dtype=torch.long))

def collate_fn(batch, pad_word_index, pad_tag_index):
    """
    Hàm tùy chỉnh để xử lý các câu có độ dài khác nhau trong một batch.
    """
    # Tách các tensor đầu vào (sentences) và nhãn (tags)
    sentences, tags = zip(*batch)
    
    # Đệm (padding) các tensor câu về cùng độ dài
    # padding_value = chỉ số của <PAD> token cho từ (word)
    sentences_padded = pad_sequence(sentences, 
                                    batch_first=True, 
                                    padding_value=pad_word_index)
    
    # Đệm (padding) các tensor nhãn về cùng độ dài
    # padding_value = chỉ số của <PAD> token cho nhãn (tag)
    tags_padded = pad_sequence(tags, 
                              batch_first=True, 
                              padding_value=pad_tag_index)
    
    return sentences_padded, tags_padded

# Tính toán các biến cần thiết
BATCH_SIZE = 32
PAD_WORD_INDEX = word_to_ix[PAD_TOKEN] 
PAD_TAG_INDEX = tag_to_ix[PAD_TOKEN] 

# Khởi tạo Datasets
train_dataset = POSDataset(train_data, word_to_ix, tag_to_ix, UNK_TOKEN)
dev_dataset = POSDataset(dev_data, word_to_ix, tag_to_ix, UNK_TOKEN)

# Khởi tạo collate_fn
train_collate_fn_with_padding = lambda batch: collate_fn(batch, PAD_WORD_INDEX, PAD_TAG_INDEX)
dev_collate_fn_with_padding = lambda batch: collate_fn(batch, PAD_WORD_INDEX, PAD_TAG_INDEX)

# Khởi tạo DataLoaders (Tạo ra train_loader và dev_loader)
train_loader = DataLoader(train_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=True, 
                          collate_fn=train_collate_fn_with_padding)

dev_loader = DataLoader(dev_dataset, 
                        batch_size=BATCH_SIZE, 
                        shuffle=False, 
                        collate_fn=dev_collate_fn_with_padding)

print("--- KẾT THÚC TASK 2 (Đã khởi tạo DataLoader) ---")

--- KẾT THÚC TASK 2 (Đã khởi tạo DataLoader) ---


In [5]:
import torch.nn as nn

# --- CÁC THAM SỐ CHO MÔ HÌNH ---

EMBEDDING_DIM = 100 
HIDDEN_DIM = 128   


VOCAB_SIZE = len(word_to_ix)
NUM_TAGS = len(tag_to_ix)
PAD_WORD_INDEX = word_to_ix[PAD_TOKEN]
PAD_TAG_INDEX = tag_to_ix[PAD_TOKEN]

class SimpleRNNForTokenClas(nn.Module):
    """
    Mô hình RNN đơn giản cho bài toán Part-of-Speech Tagging.
    """
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_tags):
        super(SimpleRNNForTokenClas, self).__init__()
        
        # 1. nn.Embedding: Chuyển chỉ số từ thành vector nhúng
        # Input: (Batch_size, Seq_len) -> Output: (Batch_size, Seq_len, Embedding_dim)
        # Sử dụng PAD_WORD_INDEX cho padding_idx để các vector nhúng của <PAD> là 0.
        self.word_embeddings = nn.Embedding(
            vocab_size, 
            embedding_dim, 
            padding_idx=PAD_WORD_INDEX # Chỉ số padding của từ
        ) 
        
        # 2. nn.RNN: Xử lý chuỗi vector embedding
        # Input: (Batch_size, Seq_len, Embedding_dim) 
        # Output: (Batch_size, Seq_len, Hidden_dim) (nếu batch_first=True)
        self.rnn = nn.RNN(
            embedding_dim,      # Kích thước đầu vào (bằng embedding_dim)
            hidden_dim,         # Kích thước trạng thái ẩn
            batch_first=True    # Đảm bảo đầu vào/ra theo định dạng (Batch, Seq, Feature)
        )
        
        # 3. nn.Linear: Ánh xạ output của RNN sang không gian nhãn
        # Input: (Batch_size, Seq_len, Hidden_dim)
        # Output: (Batch_size, Seq_len, Num_tags) - Đây là logits (raw scores)
        self.hidden2tag = nn.Linear(hidden_dim, num_tags)
        
    def forward(self, sentence):
        # 1. Embedding Layer
        embeds = self.word_embeddings(sentence)
        # embeds.shape: (Batch_size, Seq_len, Embedding_dim)
        
        # 2. RNN Layer
        # rnn_out: Output của RNN cho mỗi time step
        # h_n: Trạng thái ẩn cuối cùng (sẽ không dùng trong Token Classification)
        rnn_out, h_n = self.rnn(embeds)
        # rnn_out.shape: (Batch_size, Seq_len, Hidden_dim)
        
        # 3. Linear Layer
        tag_logits = self.hidden2tag(rnn_out)
        # tag_logits.shape: (Batch_size, Seq_len, Num_tags)
        
        return tag_logits

print("--- KẾT THÚC TASK 3 ---")
print(f"✅ Đã định nghĩa lớp SimpleRNNForTokenClas.")

--- KẾT THÚC TASK 3 ---
✅ Đã định nghĩa lớp SimpleRNNForTokenClas.


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
from torch.nn.utils.rnn import pad_sequence 

# --- KHỞI TẠO MÔ HÌNH VÀ THAM SỐ ---

# Tham số mô hình (từ Task 3)
EMBEDDING_DIM = 100 
HIDDEN_DIM = 128   

# Khởi tạo thiết bị (Device)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tham số huấn luyện
LEARNING_RATE = 0.001
NUM_EPOCHS = 10 

# Khởi tạo Mô hình (Sử dụng các biến đã định nghĩa ở Task 3)
model = SimpleRNNForTokenClas(
    VOCAB_SIZE, 
    EMBEDDING_DIM, 
    HIDDEN_DIM, 
    NUM_TAGS
).to(device)

# 1. Loss Function (Hàm mất mát)
loss_function = nn.CrossEntropyLoss(ignore_index=PAD_TAG_INDEX)

# 2. Optimizer (Bộ tối ưu hóa)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


# --- HÀM HỖ TRỢ (TÍNH ACCURACY) ---
def calculate_accuracy(predictions, targets, pad_index):
    """Tính toán độ chính xác, loại bỏ các token padding."""
    
    max_predictions = predictions.argmax(dim=-1)
    non_pad_elements = (targets != pad_index).nonzero(as_tuple=True)
    
    correct = max_predictions[non_pad_elements].eq(targets[non_pad_elements]).sum()
    total = non_pad_elements[0].size(0)
    
    if total == 0:
        return 0.0, 0
    
    accuracy = correct.float() / total
    return accuracy.item(), total


# --- VÒNG LẶP HUẤN LUYỆN (Training Loop) ---

print("--- BẮT ĐẦU TASK 4: HUẤN LUYỆN MÔ HÌNH ---")

# Vòng lặp Huấn luyện chính
for epoch in range(NUM_EPOCHS):
    model.train() 
    total_loss = 0
    total_tokens = 0
    
    for batch_idx, (words, tags) in enumerate(train_loader): # Dựa vào train_loader từ Task 2
        words, tags = words.to(device), tags.to(device)
        
        # 1. Xóa gradient cũ
        optimizer.zero_grad()
        
        # 2. Forward pass
        output = model(words)
        
        # Chuyển đổi định dạng output và tags để tính CrossEntropyLoss
        output = output.view(-1, output.shape[-1])
        tags = tags.view(-1)
        
        # 3. Tính loss
        loss = loss_function(output, tags)
        
        # 4. Backward pass
        loss.backward()
        
        # 5. Cập nhật trọng số
        optimizer.step()
        
        # Cập nhật tổng loss
        total_loss += loss.item() * tags.shape[0] 
        total_tokens += tags.shape[0]
        
    avg_loss = total_loss / total_tokens
    
    # In ra giá trị loss trung bình sau mỗi epoch
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} | Loss trung bình: {avg_loss:.4f}")

print("--- KẾT THÚC TASK 4 ---")

--- BẮT ĐẦU TASK 4: HUẤN LUYỆN MÔ HÌNH ---
Epoch 1/10 | Loss trung bình: 1.0537
Epoch 2/10 | Loss trung bình: 0.5766
Epoch 3/10 | Loss trung bình: 0.4315
Epoch 4/10 | Loss trung bình: 0.3470
Epoch 5/10 | Loss trung bình: 0.2848
Epoch 6/10 | Loss trung bình: 0.2399
Epoch 7/10 | Loss trung bình: 0.2040
Epoch 8/10 | Loss trung bình: 0.1753
Epoch 9/10 | Loss trung bình: 0.1506
Epoch 10/10 | Loss trung bình: 0.1294
--- KẾT THÚC TASK 4 ---


In [7]:
def evaluate(model, data_loader, loss_function, pad_index, device):
    """
    Đánh giá hiệu năng của mô hình trên tập dữ liệu.
    """
    # Đặt mô hình ở chế độ đánh giá 
    model.eval() 
    
    total_correct = 0
    total_tokens_count = 0
    total_loss = 0
    
    # Tắt việc tính toán gradient 
    with torch.no_grad():
        for words, tags in data_loader:
            words, tags = words.to(device), tags.to(device)
            
            # Forward pass
            output = model(words)
            
            # Tính loss (cần cho việc theo dõi)
            output_loss = output.view(-1, output.shape[-1])
            tags_loss = tags.view(-1)
            loss = loss_function(output_loss, tags_loss)
            total_loss += loss.item() * tags_loss.shape[0]
            
            # Tính Accuracy
            # Áp dụng torch.argmax trên chiều cuối cùng của output để lấy dự đoán 
            accuracy, tokens_count = calculate_accuracy(output, tags, pad_index)
            
            total_correct += int(accuracy * tokens_count)
            total_tokens_count += tokens_count
            
    avg_loss = total_loss / total_tokens_count
    
    # Độ chính xác chỉ tính trên các token không phải padding 
    final_accuracy = total_correct / total_tokens_count if total_tokens_count > 0 else 0.0
    
    return final_accuracy, avg_loss

In [8]:
# --- VÒNG LẶP HUẤN LUYỆN (Tích hợp Đánh giá) ---

print("\n--- BẮT ĐẦU HUẤN LUYỆN và ĐÁNH GIÁ  ---")

best_dev_accuracy = 0.0
best_model_state = None

for epoch in range(NUM_EPOCHS):
    start_time = time.time()
    
    # --- Giai đoạn Huấn luyện (Train) ---
    model.train() 
    total_loss = 0
    total_tokens = 0
    total_correct_train = 0
    
    for words, tags in train_loader:
        words, tags = words.to(device), tags.to(device)
        optimizer.zero_grad()
        output = model(words)
        
        # Tính loss
        output_flat = output.view(-1, output.shape[-1])
        tags_flat = tags.view(-1)
        loss = loss_function(output_flat, tags_flat)
        loss.backward()
        optimizer.step()
        
        # Cập nhật metrics huấn luyện
        total_loss += loss.item() * tags_flat.shape[0]
        accuracy_train, tokens_count = calculate_accuracy(output, tags, PAD_TAG_INDEX)
        total_correct_train += int(accuracy_train * tokens_count)
        total_tokens += tokens_count
        
    avg_train_loss = total_loss / total_tokens
    avg_train_accuracy = total_correct_train / total_tokens
    
    # --- Giai đoạn Đánh giá (Dev) ---
    dev_accuracy, avg_dev_loss = evaluate(model, dev_loader, loss_function, PAD_TAG_INDEX, device)
    
    # Báo cáo kết quả sau mỗi epoch 
    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS} (Time: {time.time() - start_time:.2f}s):")
    print(f"  [TRAIN] Loss: {avg_train_loss:.4f} | Accuracy: {avg_train_accuracy*100:.2f}%")
    print(f"  [DEV]   Loss: {avg_dev_loss:.4f} | Accuracy: {dev_accuracy*100:.2f}%")
    
    # Lựa chọn mô hình tốt nhất dựa trên độ chính xác tập dev 
    if dev_accuracy > best_dev_accuracy:
        best_dev_accuracy = dev_accuracy
        best_model_state = model.state_dict()
        print(f"  *Mô hình DEV tốt nhất được cập nhật: {best_dev_accuracy*100:.2f}%*")

# Lưu mô hình tốt nhất
if best_model_state:
    torch.save(best_model_state, 'best_pos_rnn_model.pt')

print("\n--- KẾT THÚC HUẤN LUYỆN và ĐÁNH GIÁ ---")
print(f"Độ chính xác cuối cùng trên tập dev (tốt nhất): {best_dev_accuracy*100:.2f}%")


--- BẮT ĐẦU HUẤN LUYỆN và ĐÁNH GIÁ  ---

Epoch 1/10 (Time: 2.20s):
  [TRAIN] Loss: 0.3493 | Accuracy: 96.31%
  [DEV]   Loss: 1.4303 | Accuracy: 87.85%
  *Mô hình DEV tốt nhất được cập nhật: 87.85%*

Epoch 2/10 (Time: 1.83s):
  [TRAIN] Loss: 0.3027 | Accuracy: 96.84%
  [DEV]   Loss: 1.4790 | Accuracy: 88.08%
  *Mô hình DEV tốt nhất được cập nhật: 88.08%*

Epoch 3/10 (Time: 1.81s):
  [TRAIN] Loss: 0.2639 | Accuracy: 97.27%
  [DEV]   Loss: 1.5426 | Accuracy: 87.94%

Epoch 4/10 (Time: 1.81s):
  [TRAIN] Loss: 0.2273 | Accuracy: 97.66%
  [DEV]   Loss: 1.5930 | Accuracy: 87.87%

Epoch 5/10 (Time: 1.83s):
  [TRAIN] Loss: 0.1955 | Accuracy: 97.99%
  [DEV]   Loss: 1.6623 | Accuracy: 87.79%

Epoch 6/10 (Time: 1.82s):
  [TRAIN] Loss: 0.1699 | Accuracy: 98.27%
  [DEV]   Loss: 1.7088 | Accuracy: 87.82%

Epoch 7/10 (Time: 1.80s):
  [TRAIN] Loss: 0.1454 | Accuracy: 98.52%
  [DEV]   Loss: 1.7773 | Accuracy: 87.87%

Epoch 8/10 (Time: 1.82s):
  [TRAIN] Loss: 0.1251 | Accuracy: 98.72%
  [DEV]   Loss: 1.8

In [9]:
# Tạo từ điển đảo ngược (index_to_tag) để chuyển chỉ số sang nhãn UPOS
index_to_tag = {v: k for k, v in tag_to_ix.items()}

def predict_sentence(sentence_str, model, word_to_ix, index_to_tag, unk_token, device):
    """
    Nhận vào một câu dạng chuỗi, xử lý, dự đoán nhãn POS và in ra các cặp (từ, nhãn_dự đoán).
    """
    model.eval()
    
    # 1. Tiền xử lý: Tách từ và chuyển thành chỉ số
    words = sentence_str.split()
    indices = [get_index_from_word(word, word_to_ix, unk_token) for word in words]
    
    # 2. Chuyển thành tensor và đẩy lên device
    sentence_tensor = torch.tensor(indices, dtype=torch.long).unsqueeze(0).to(device) # (1, Seq_len)
    
    with torch.no_grad():
        # 3. Forward pass
        output = model(sentence_tensor) # (1, Seq_len, Num_tags)
        
        # 4. Lấy chỉ số nhãn dự đoán
        predicted_tags_indices = output.squeeze(0).argmax(dim=1).cpu().numpy()
        
    # 5. Ánh xạ chỉ số sang nhãn (tag)
    predicted_tags = [index_to_tag[idx] for idx in predicted_tags_indices]
    
    # 6. In kết quả (từ, nhãn_dự đoán)
    result = [(word, tag) for word, tag in zip(words, predicted_tags)]
    
    return result

# --- Ví dụ Dự đoán ---
print("\n--- KẾT QUẢ DỰ ĐOÁN CÂU MỚI (Task 5 Nâng cao) ---")
test_sentence = "I love NLP and PyTorch" 
predictions = predict_sentence(test_sentence, model, word_to_ix, index_to_tag, UNK_TOKEN, device)

print(f"Câu: \"{test_sentence}\"")
print(f"Dự đoán: {predictions}") 
print("-----------------------------------------------------")


--- KẾT QUẢ DỰ ĐOÁN CÂU MỚI (Task 5 Nâng cao) ---
Câu: "I love NLP and PyTorch"
Dự đoán: [('I', 'PRON'), ('love', 'VERB'), ('NLP', 'PROPN'), ('and', 'CCONJ'), ('PyTorch', 'ADV')]
-----------------------------------------------------
